# Spark ML Production

## Загружаем данные

In [9]:
import $ivy.`org.apache.spark:spark-sql_2.12:3.0.1`
import $ivy.`org.apache.spark:spark-mllib_2.12:3.0.1`
import $ivy.`sh.almond:almond-spark_2.12:0.10.9`

import $ivy.$                                      

import $ivy.$                                        

import $ivy.$                                   

In [10]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [11]:
import org.apache.spark.sql._

val spark = {
      SparkSession.builder()
        .master("local[*]")
        .getOrCreate()
    }

import spark.implicits._

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@229d7f5
import spark.implicits._

In [12]:
val data = spark
        .read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv("BankChurners.csv")

data: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 21 more fields]

## Загружаем модель

In [13]:
import org.apache.spark.ml.PipelineModel

val model = PipelineModel.load("pipelineModel")

import org.apache.spark.ml.PipelineModel


model: PipelineModel = pipeline_ce9e0360a2e0

## Вычисляем

In [14]:
val prediction = model.transform(data)
prediction.show(10)

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|CLIENTNUM|   Attriti

|810347208|Existing Customer|          51|     M|              4|        Unknown|       Married|        $120K +|         Gold|            46|                       6|                     1|                    3|     34516.0|               2264|        32252.0|               1.975|           1330|            31|              0.722|                0.066|                                                                                                                         1.2303E-4|                                                                                                                           0.99988|                   0.0|                    5.0|           1.0|                    2.0|                  2.0|                    (5,[],[])|               (3,[0],[1.0])|           (1,[],[])|                (6,[2],[1.0])|              (3,[2],[1.0])|(28,[0,1,2,3,4,5,...|(28,[0,1,2,3,4,5,...|(10,[0,1,2,3,4,5,...|[1.35336642426477...|[0.79467945243139...|       0.0|
|818906208|Existing C

prediction: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 37 more fields]

## Проверяем результат

In [15]:
val tp = prediction.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 1)).count
val tn = prediction.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 0)).count
val fp = prediction.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 1)).count
val fn = prediction.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 0)).count

println(s"Confusion Matrix:\n$tp\t$fp\n$fn\t$tn\n")

Confusion Matrix:
1206	1924
421	6576



tp: Long = 1206L
tn: Long = 6576L
fp: Long = 1924L
fn: Long = 421L

In [16]:
val accuracy = (tp + tn) / (tp + tn + fp + fn).toDouble
val precision = tp / (tp + fp).toDouble
val recall = tp / (tp + fn).toDouble

println(s"Accuracy = $accuracy")
println(s"Precision = $precision")
println(s"Recall = $recall\n")

Accuracy = 0.7684408018169251
Precision = 0.3853035143769968
Recall = 0.7412415488629379



accuracy: Double = 0.7684408018169251
precision: Double = 0.3853035143769968
recall: Double = 0.7412415488629379